
TITLE
=====

##### Autori: Giovanni Bitonti, Ana Pascual


In [1]:
import os
import pandoc
import numpy as np
import threading as thr
import matplotlib.pyplot as plt
from keras.models import Sequential
from keras.layers import Conv2D, BatchNormalization, MaxPool2D, Dense, Flatten, InputLayer, Activation, Dropout
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_curve, auc
from keras.callbacks import ModelCheckpoint, ReduceLROnPlateau, EarlyStopping

In [2]:
import matlab.engine
eng = matlab.engine.start_matlab()

In [6]:
mammo_o = []
mammo_f = []
label = []
project_folder = "C:/Users/anapascual/exam_project/dataset/"
os.chdir(project_folder)
l = os.listdir()

In [7]:
def create_dataset(ls, o_img, f_img, lbl):
    """Function calling the Matlab file in order to filter the images
    
    ARGUMENTS
    ---------
    ls : list
         Chunk of files' directory.
         
    
    Return:
        Dataset with all images filtered.
    """
    if "_1_resized.pgm" in l:
        mo, mf = eng.dataset_filtered(eng.char(os.path.join(project_folder,l)), nargout = 2)
        o_img.append(mo)
        f_img.append(mf)
        lbl.append(1)
    elif "_2_resized.pgm" in l:
        mo, mf = eng.dataset_filtered(eng.char(os.path.join(project_folder,l)), nargout = 2)
        o_img.append(mo)
        f_img.append(mf)
        lbl.append(0)

In [8]:
os.chdir("C:/Users/anapascual/exam_project/")
threads = []
chunk = 49
print(chunk)
for i in range(6):
    t = thr.Thread(target = create_dataset, args = (l[i*chunk : (i+1)*chunk], mammo_o, mammo_f, label))
    threads.append(t)
    t.start()

for i in threads:
    i.join()

49


In [9]:
eng.quit()

In [10]:
mammo_o = np.asarray(mammo_o, dtype = 'float32')/255.
mammo_f = np.asarray(mammo_f, dtype = 'float32')/255.
label = np.asarray(label)

In [11]:
mammo_o_4d = np.reshape(mammo_o, (147, 125, 125, 1))
print(mammo_o_4d.shape)
mammo_f_4d = np.reshape(mammo_f, (147, 64, 64, 1))
print(mammo_f_4d.shape)

ValueError: cannot reshape array of size 0 into shape (147,125,125,1)

In [ ]:
def cnn_o(shape=(125, 125, 1)):
    model = Sequential([
        
        Conv2D(5, (5,5), padding = 'same', input_shape = shape),
        BatchNormalization(),
        Activation('relu'),
        
        MaxPool2D((6,6), strides = 2),
        #Dropout(0.2),
        
        
        Conv2D(6, (5,5), padding = 'same'),
        BatchNormalization(),
        Activation('relu'),
        
        MaxPool2D((6,6), strides = 2),
        #Dropout(0.1),
        
        
        Conv2D(10, (5,5), padding = 'same'),
        BatchNormalization(),
        Activation('relu'),
        
        MaxPool2D((6,6), strides = 2),
        #Dropout(0.1),
        
        Flatten(),
        
        Dense(10, activation = 'relu'),
        #Dropout(0.1),
        Dense(1, activation = 'sigmoid')        
        
    ])
    return model

In [ ]:
model_o = cnn_o()
model_o.summary()

In [ ]:
from tensorflow.keras.optimizers import SGD
learning_rate = 0.001
model_o.compile(optimizer = SGD(learning_rate, momentum = 0.9), loss = 'binary_crossentropy', metrics = ['accuracy'])
#model_f.compile(optimizer = SGD(learning_rate, momentum = 0.9), loss = 'binary_crossentropy', metrics = ['accuracy'])

In [ ]:
from keras.preprocessing.image import ImageDataGenerator

aug = ImageDataGenerator(
                rotation_range = 90,
                horizontal_flip = True,
                vertical_flip = True,
                validation_split = 0.21)

Xaug_train_o = aug.flow(X_slice_o, Y_slice_o, batch_size = 30, subset = 'training')
Xaug_val_o = aug.flow(X_slice_o, Y_slice_o, batch_size = 30, subset = 'validation')

Xaug_train_f = aug.flow(X_slice_f, Y_slice_f, subset = 'training')
Xaug_val_f = aug.flow(X_slice_f, Y_slice_f, subset = 'validation')

In [ ]:
reduce_on_plateau = ReduceLROnPlateau(
    monitor="val_loss",
    factor=0.1,
    patience=10,
    verbose=0,
    mode="auto",
    min_delta=0.0001,
    cooldown=0,
    min_lr=0)

In [ ]:
X_train_o, X_val_o, Y_train_o, Y_val_o = train_test_split(mammo_o_4d, label, test_size = 0.2)
batch_size = 49
traino = model_o.fit(X_train_o, Y_train_o,
                            batch_size = batch_size,
                            epochs = 200,
                            #steps_per_epoch = np.shape(X_train_o)[0]/batch_size,
                            verbose=1,
                            validation_data=(X_val_o, Y_val_o), callbacks = [reduce_on_plateau])#,
                            #validation_steps=np.shape(X_val_o)[0]/batch_size,
                            #callbacks=[checkpoint,reduce_on_plateau])
    

In [ ]:
acc = traino.history['accuracy']
val_acc = traino.history['val_accuracy']
loss = traino.history['loss']
val_loss = traino.history['val_loss']
    
epochs_range = range(1, len(acc)+1)
    #Train and validation accuracy 
plt.figure(figsize=(15, 15))
plt.subplot(2, 2, 1)
plt.plot(epochs_range, acc, label='Training Accuracy')
plt.plot(epochs_range, val_acc, label='Validation Accuracy')
plt.legend(loc='lower right')
plt.title('Training and Validation Accuracy')
    #Train and validation loss 
plt.subplot(2, 2, 2)
plt.plot(epochs_range, loss, label='Training Loss')
plt.plot(epochs_range, val_loss, label='Validation Loss')
plt.legend(loc='upper right')
plt.title('Training and Validation Loss')
plt.show()

In [ ]:
_, val_acc = model_o.evaluate(X_val_o, Y_val_o, verbose=0)
print('Validation accuracy: %.3f' % (val_acc))
    
    
preds = model_o.predict(X_val_o, verbose=1)

    #Compute Receiver operating characteristic (ROC)
fpr, tpr, _ = roc_curve(Y_val_o, preds)

roc_auc = auc(fpr, tpr)
   #Plot of a ROC curve
plt.figure()
lw = 2
plt.plot(fpr, tpr, color='darkorange', lw=lw, label='ROC curve (area = %0.2f)' % roc_auc)
plt.plot([0, 1], [0, 1], color='navy', lw=lw, linestyle='--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver operating characteristic example')
plt.legend(loc="lower right")
plt.show()

In [ ]:
#CROSS-VALIDATION with ORIGINAL DATA (X_slice_o)

A = []
V_A = []
L = []
V_L = []
ROC = []

for i in range(3):
    X_train_o, X_val_o, Y_train_o, Y_val_o = train_test_split(mammo_o_4d, label, test_size = 0.168, random_state = 40)
    traino = model_o.fit(X_train_o, Y_train_o,
                            batch_size = 49,
                            epochs = 200,
                            verbose=1,
                            validation_data=(X_val_o, Y_val_o),
                            callbacks=[checkpoint,reduce_on_plateau])
    
    acc = traino.history['accuracy']
    val_acc = traino.history['val_accuracy']
    loss = traino.history['loss']
    val_loss = traino.history['val_loss']
    
    A.append(acc)
    V_A.append(val_acc)
    L.append(loss)
    V_L.append(val_loss)
    
    epochs_range = range(1, len(acc)+1)
    #Train and validation accuracy 
    plt.figure(figsize=(15, 15))
    plt.subplot(2, 2, 1)
    plt.plot(epochs_range, acc, label='Training Accuracy')
    plt.plot(epochs_range, val_acc, label='Validation Accuracy')
    plt.legend(loc='lower right')
    plt.title('Training and Validation Accuracy')
    #Train and validation loss 
    plt.subplot(2, 2, 2)
    plt.plot(epochs_range, loss, label='Training Loss')
    plt.plot(epochs_range, val_loss, label='Validation Loss')
    plt.legend(loc='upper right')
    plt.title('Training and Validation Loss')
    plt.show()
    
    _, val_acc = model_o.evaluate(X_val_o, Y_val_o, verbose=0)
    print('Validation accuracy: %.3f' % (val_acc))
    
    
    preds = model_o.predict(X_val_o, verbose=1)

    #Compute Receiver operating characteristic (ROC)
    fpr, tpr, _ = roc_curve(Y_val_o, preds)

    roc_auc = auc(fpr, tpr)
    ROC.append(roc_auc)
    #Plot of a ROC curve
    plt.figure()
    lw = 2
    plt.plot(fpr, tpr, color='darkorange', lw=lw, label='ROC curve (area = %0.2f)' % roc_auc)
    plt.plot([0, 1], [0, 1], color='navy', lw=lw, linestyle='--')
    plt.xlim([0.0, 1.0])
    plt.ylim([0.0, 1.05])
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    plt.title('Receiver operating characteristic example')
    plt.legend(loc="lower right")
    plt.show()

In [ ]:
media = np.mean(ROC)
std = np.std(ROC)
print(media)
print(std)